# Learning CVXPY
This is a python package that is used to solve convex optimization problems. A optimization problem is when you have a action (e.g. rescoruse allocation), some constraits (e.g. Number needed) and a outcome. This method can be applied to alot of problems however almost all are unsolvable, with a exection being convex problems. A convex problem is when the constraints are linear, and the outcome has none negative curvature.     
A simple optimization problem is shown below

## What is CVXPY?
A simple optimization problem is shown below

In [1]:
import cvxpy as cp

# This creats 2 scalar varibles 
x = cp.Variable()
y = cp.Variable()

#This defines the containts that we want to use for this problem
constraints = [x + y == 1,
               x - y >= 1]

#This defines the objective of the problem
obj = cp.Minimize((x - y)**2)

#Solves the problem we have defined
prob = cp.Problem(obj, constraints)
prob.solve()
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", x.value, y.value)

status: optimal
optimal value 1.0
optimal var 1.0 1.570086213240983e-22


The status of this problem is shown as optimal meaning that the problem was solved correctly. The optimal value is the lowest soluation to the objective and final the optimal var is the values of x and y that gives the lowest solution.

If we wanted to have a different problem we would have to define a new problem as they are immutable.

In [3]:
prob2 = cp.Problem(cp.Maximize(x + y), prob.constraints)
print("optimal value ", prob2.solve())

#This replaces the first constraint defined abouve
constraints = [x + y <= 3] + prob2.constraints[1:0]
prob3 = cp.Problem(prob2.objective, constraints)
print("optimal value ", prob3.solve())

optimal value  0.9999999999945575
optimal value  2.999999999735974


If once you try and solve a problem and get the status infeasible (not possible) or unbounded (no constraints) means that I can not be solved.

In [5]:
x = cp.Variable()

prob = cp.Problem(cp.Minimize(x),[x >= 1, x <=0])
prob.solve()
print("status:",prob.status)
print("optimal value", prob.value)

prob = cp.Problem(cp.Minimize(x))
prob.solve()
print("status:", prob.status)
print("optimal value", prob.value)

status: infeasible
optimal value inf
status: unbounded
optimal value -inf


It should be noted that the values are oposite for maximization problems.

There are a number of other staties that can be shown in this. For instance is it can solve the problem but to a lower accuracy than desired then the status will indicate as such, "optimal_inaccurate".           
The solver could also fail to solve the problem enteriley and then you would get SolverError for the status.

### Vectors and matrices
The varibles used in problems done have to be scalar, you could have vector and matrices as well. You also dont have to use cpy's varible function for arrays, it also surports NumPy ndarrays, NumPy matrices and SciPy sparse matrices. 

In [6]:
# A scalar variable.
a = cp.Variable()
x = cp.Variable(5)
x = cp.Variable((5, 1))
A = cp.Variable((4, 7))

import numpy as np
np.random.seed(1)
A = np.random.randn(10, 5)
b = np.random.randn(10)

x = cp.Variable(5)
objective = cp.Minimize(cp.sum_squares(A @ x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

print("Optimal value", prob.solve())
print("Optimal var")
print(x.value) # A numpy ndarray.

Optimal value 4.141338603672535
Optimal var
[-4.95922264e-21  6.07571976e-21  1.34643668e-01  1.24976681e-01
 -4.57130806e-21]


You notice that the constraints are written either as ==, <= and >=. These are all the constraints that are permitted and will me applied to each element of the matrix. You are also able to do semi-definite cone constraints but that will be done later. Should also be noted that you can not chain constraints together.

### Parameters 
Parameters are used to change the value of constants in a problem without needing to reconstruct the whole problem. This is because parametrizing the problem multiple times and solving them is normally fasting the solving the whole problem as once.            
Parameters can be vectors or matrices, you can also specify is there parameters entries are negative or symmetric. You can also assign them a value at any point.

In [8]:
m = cp.Parameter(nonneg=True)

c = cp.Parameter(5)

G = cp.Parameter((4, 7), nonpos=True)

G.value = -np.ones((4, 7))

rho = cp.Parameter(nonneg=True)
rho.value = 2

rho = cp.Parameter(nonneg=True, value=2)

## Disciplined Convex Programming
This is a system that constructs expressions with known curvature from a given library of base functions, this helps ensure that problems are convex.

### Expressions 
These are formed from vaiables, parameters, constants amoung other things. Some examples are shown.

In [ ]:
# Create variables and parameters.
x, y = cp.Variable(), cp.Variable()
a, b = cp.Parameter(), cp.Parameter()

# Examples of CVXPY expressions.
3.69 + b/3
x - 4*a
sqrt(x) - minimum(y, x - a)
maximum(2.66 - sqrt(y), square(x + 2*y))

The expressions can be scalars, vectors or matrices and the shape can be accessed by `expr.shape`. CVXPY will raise a error if the expressions dont make sence, e.g. adding matrices of different sizes. 

In [11]:
X = cp.Variable((5, 4))
A = np.ones((3, 5))

# Use expr.shape to get the dimensions.
print("dimensions of X:", X.shape)
print("size of X:", X.size)
print("number of dimensions:", X.ndim)
print("dimensions of sum(X):", cp.sum(X).shape)
print("dimensions of A @ X:", (A @ X).shape)

# ValueError raised for invalid dimensions.
try:
    A + X
except ValueError:
    print('Problem')

dimensions of X: (5, 4)
size of X: 20
number of dimensions: 2
dimensions of sum(X): ()
dimensions of A @ X: (3, 4)
Problem


#### Curvature
Each (sub)expression is flagged as one of the following
* constant
* affine
* convex
* concave
* unknown
There are a number of rules of what the expressions curvature will be based on the subexpressions. It is also stored in the extrestion `expr.curvature`. The curvature rules are can be found in https://www.cvxpy.org/tutorial/dcp/index.html   